In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.8 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp310-cp310-linux_x86_64.whl size=661204 sha256=46b46ad71b0be38e7c00bbed4cda7c5c7340242d8b72d976844e06b9b032dac9
  Stored in directory: /root/.cache/pip/wheels/12/34/d2/9a349255a4eca3a486d82c79d21e138ce2ccd90f414d9d72b8
Successfully built pycuda


In [4]:
%%writefile array_addition_pycuda.py
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# CUDA kernel code for array addition
cuda_code = """
__global__ void add_arrays(int *a, int *b, int *c, int size) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;

    if (tid < size) {
        c[tid] = a[tid] + b[tid];
    }
}
"""

# Host data
a_host = np.array([1, 2, 3], dtype=np.int32)
b_host = np.array([4, 5, 6], dtype=np.int32)
size = len(a_host)

# Device data
a_device = cuda.mem_alloc(a_host.nbytes)
b_device = cuda.mem_alloc(b_host.nbytes)
c_device = cuda.mem_alloc(a_host.nbytes)

# Copy data to device
cuda.memcpy_htod(a_device, a_host)
cuda.memcpy_htod(b_device, b_host)

# Load the CUDA module
cuda_module = SourceModule(cuda_code)
add_arrays_kernel = cuda_module.get_function("add_arrays")

# Set up block and grid dimensions
block_dim = (size, 1, 1)
grid_dim = (1, 1)

# Launch the CUDA kernel
add_arrays_kernel(a_device, b_device, c_device, np.int32(size), block=block_dim, grid=grid_dim)

# Copy the result back to the host
c_host = np.empty_like(a_host)
cuda.memcpy_dtoh(c_host, c_device)

# Display the result
print("Array A: ", a_host)
print("Array B: ", b_host)
print("Result Array C: ", c_host)



Overwriting array_addition_pycuda.py


In [5]:
!python array_addition_pycuda.py

Array A:  [1 2 3]
Array B:  [4 5 6]
Result Array C:  [5 7 9]
